In [0]:
%sql
drop database if exists dev.swatch_consumption cascade;

In [0]:
class SetupGoldLayer:
    def __init__(self):
        self.catalog_name = "dev"
        self.gold_layer_schema_name = "swatch_consumption"
        self.gym_summary_table_name = "gym_summary"
        self.workout_bpm_summary_table_name = "workout_bpm_summary"

In [0]:
class CreateGoldLayerTables(SetupGoldLayer):
    def __init__(self):
        super().__init__()
    
    def create_gold_layer_schema(self):
        spark.sql(f"create schema if not exists {self.catalog_name}.{self.gold_layer_schema_name}")
    
    def create_gym_summary_table(self):
        spark.sql(f""" 
                  create table if not exists {self.catalog_name}.{self.gold_layer_schema_name}.{self.gym_summary_table_name}
                  (
                    gym_id string,
                    mac_address timestamp,
                    `date` date,
                    workouts string,
                    minutes_in_gym double,
                    minutes_exercising double
                  ) using delta
                  """)
        print(f"Table Created - {self.catalog_name}.{self.gold_layer_schema_name}.{self.gym_summary_table_name}")

    def create_workout_bpm_summary_table(self):
        spark.sql(f""" 
                  create table if not exists {self.catalog_name}.{self.gold_layer_schema_name}.{self.workout_bpm_summary_table_name}
                  (
                      user_id string,
                      `date` date,
                      workout_id string,
                      session_id int,
                      sex string,
                      city string,
                      state string,
                      recordings int,
                      max_hr double,
                      min_hr double,
                      avg_hr double
                  ) using delta
                  """)
        
        print(f"Table Created - {self.catalog_name}.{self.gold_layer_schema_name}.{self.workout_bpm_summary_table_name}")
    
    
    def create_tables(self):
        self.create_gold_layer_schema()
        self.create_gym_summary_table()
        self.create_workout_bpm_summary_table()


In [0]:
# Create the checkpoint volumes
class  CreateGoldLayerCheckpointDirs(SetupGoldLayer):
    def __init__(self):
        super().__init__()

    def create_gym_summary_table_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.gold_layer_schema_name}.{self.gym_summary_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/gym_summary/' ")
                
        print(f"Checkpoint Volume Created - {self.catalog_name}.{self.gold_layer_schema_name}.{self.gym_summary_table_name}_checkpoint")
    
    def create_workout_bpm_summary_table_checkpoint(self):
        spark.sql(f"create external volume if not exists {self.catalog_name}.{self.gold_layer_schema_name}.{self.workout_bpm_summary_table_name}_checkpoint location 'abfss://smart-watch-analytics@dbstorageact.dfs.core.windows.net/swatch-checkpoint/workout_bpm_summary/' ")
                
        print(f"Checkpoint Volume Created - {self.catalog_name}.{self.gold_layer_schema_name}.{self.workout_bpm_summary_table_name}_checkpoint")


    def create_gold_layer_checkpoint_volumes(self):
        self.create_gym_summary_table_checkpoint()
        self.create_workout_bpm_summary_table_checkpoint()
        print("Gold layer checkpoint volumes created successfully ! \n")

In [0]:
tables = CreateGoldLayerTables()
tables.create_tables()

checkpoints = CreateGoldLayerCheckpointDirs()
checkpoints.create_gold_layer_checkpoint_volumes()

Table Created - dev.swatch_consumption.gym_summary
Table Created - dev.swatch_consumption.workout_bpm_summary
Checkpoint Volume Created - dev.swatch_consumption.gym_summary_checkpoint
Checkpoint Volume Created - dev.swatch_consumption.workout_bpm_summary_checkpoint
Gold layer checkpoint volumes created successfully ! 

